In [14]:
#installed.packages()['devtools']

[1] NA

In [1]:
install.packages("devtools")

also installing the dependencies ‘credentials’, ‘zip’, ‘gitcreds’, ‘ini’, ‘gert’, ‘gh’, ‘rex’, ‘crosstalk’, ‘rlang’, ‘cachem’, ‘xopen’, ‘brew’, ‘usethis’, ‘covr’, ‘DT’, ‘memoise’, ‘rcmdcheck’, ‘remotes’, ‘roxygen2’, ‘rversions’, ‘sessioninfo’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
install.packages("remotes")
remotes::install_github("mkuhn/dict")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done




BH   (NA    -> 1.75.0-0) [CRAN]
Rcpp (1.0.5 -> 1.0.6   ) [CRAN]


Installing 2 packages: BH, Rcpp

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



✔  checking for file ‘/tmp/RtmpbrWCqw/remotes212b31eb314b/mkuhn-dict-f498319/DESCRIPTION’
─  preparing ‘dict’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  installing the package to process help pages
─  saving partial Rd database (17.2s)
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘dict_0.10.0.tar.gz’
   


In [3]:
install.packages("sjmisc")

also installing the dependencies ‘insight’, ‘sjlabelled’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [4]:
library(data.table)
library(magrittr)
library(dplyr)
library(tidyr)
library(sjmisc)
library(dict)
library(stats)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following object is masked from ‘package:magrittr’:

    extract



Attaching package: ‘sjmisc’


The following object is masked from ‘package:tidyr’:

    replace_na




Whole Blood DF

In [5]:
fn1 = "gene_wb_formatted.csv"
wb = fread(fn1, data.table = F)

Avoidable 13.219 seconds. This file is very unusual: it ends abruptly without a final newline, and also its size is a multiple of 4096 bytes. Please properly end the last row with a newline using for example 'echo >> file' to avoid this  time to copy.


Warning message in fread(fn1, data.table = F):
“Discarded single-line footer: <<"GSM1008317","SUB134244",63.67,"Male","White","SDY212",16.04552164,11.12499688,7.66274171,9.3022515535,13.71108939,7.233837263,7.693793282,7.420161509,7.166243439,10.220894368,7.228720584,9.720067053,7.400436316,9.068089005,7.3019745505,7.274963267,7.307868371,6.879948319,8.1217533825,11.14300304,8.3590256535,7.668949496,7.508974472,7.17319041,9.928773434,11.13384797,7.7605063975,9.911360527,7.781743288,7.021383115,7.484818867,7.500712041,7.816412293,8.548237007,9.1320352925,7.3406620935,7.48756>>”


In [6]:
drop = 'NA'
wb_drop = wb[ , !(colnames(wb) %in% drop)]

Results DF

In [7]:
### LOADING txt files
# read cmv lab test results
fn = "lab_test.txt"
lab_test = fread(fn, data.table = F)
cmv = lab_test%>%filter(NAME_REPORTED=="CMV Ab")
#View(cmv)

# read expsample to biosample
fn = "expsample_2_biosample.txt"
e2b = fread(fn, data.table = F)


# read biosample table
fn = "biosample.txt"
biosample = fread(fn, data.table = F)
biosample = biosample%>%select(SUBJECT_ACCESSION, BIOSAMPLE_ACCESSION, STUDY_ACCESSION)


# find gene expression result
fn = "expsample_public_repository.txt"
pub_repos = fread(fn, data.table = F)
pub_repos = pub_repos%>%filter(REPOSITORY_NAME=="GEO")


### JOINING TABLES
#(1) map CMV result to subject ID
cmv = cmv%>%inner_join(biosample, by="BIOSAMPLE_ACCESSION")


#(2) map gene expression result to biosample w/ expsample_accession
pub_repos1 = pub_repos%>%inner_join(e2b[, c("EXPSAMPLE_ACCESSION","BIOSAMPLE_ACCESSION")], by="EXPSAMPLE_ACCESSION")


#(3) map gene expression result to subject
pub_repos2 = pub_repos1%>%inner_join(biosample, by="BIOSAMPLE_ACCESSION")


# map gene expression with CMV result
results = cmv%>%inner_join(pub_repos2, by=c("SUBJECT_ACCESSION","STUDY_ACCESSION"))#<<== ORIGINAL JOIN
##results = cmv%>%full_join(pub_repos2, by=c("SUBJECT_ACCESSION","STUDY_ACCESSION", "BIOSAMPLE_ACCESSION"))

In [8]:
head(results)

,LAB_TEST_ACCESSION,LAB_TEST_PANEL_ACCESSION,BIOSAMPLE_ACCESSION.x,NAME_PREFERRED,NAME_REPORTED,REFERENCE_RANGE_ACCESSION,RESULT_UNIT_PREFERRED,RESULT_UNIT_REPORTED,RESULT_VALUE_PREFERRED,RESULT_VALUE_REPORTED,WORKSPACE_ID,SUBJECT_ACCESSION,STUDY_ACCESSION,RESULT_ID,EXPSAMPLE_ACCESSION,REPOSITORY_ACCESSION,REPOSITORY_NAME,BIOSAMPLE_ACCESSION.y
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,LT1075246,LP30085,BS965102,,CMV Ab,,,Titer,7.7,7.7,2883,SUB134259,SDY315,473,ES781245,GSM1573135,GEO,BS717442
2,LT1075248,LP30085,BS965103,,CMV Ab,,,Titer,1.0,1,2883,SUB134295,SDY315,466,ES781252,GSM1573160,GEO,BS717449
3,LT1075250,LP30085,BS965104,,CMV Ab,,,Titer,-0.1,-0.1,2883,SUB134325,SDY315,485,ES781269,GSM1573175,GEO,BS717466
4,LT1075252,LP30085,BS965105,,CMV Ab,,,Titer,-0.2,-0.2,2883,SUB116530,SDY315,514,ES781233,GSM1573186,GEO,BS717430
5,LT1075254,LP30085,BS965106,,CMV Ab,,,Titer,6.3,6.3,2883,SUB134256,SDY315,461,ES781228,GSM1573132,GEO,BS717425
6,LT1075256,LP30085,BS965107,,CMV Ab,,,Titer,-0.1,-0.1,2883,SUB116534,SDY315,502,ES781229,GSM1573190,GEO,BS717426


In [9]:
#res_x
names(results)[grepl("BIOSAMPLE_ACCESSION.x", names(results))] = "BIOSAMPLE_ACCESSION"

In [10]:
#dropped "BIOSAMPLE_ACCESSION.y", "BIOSAMPLE_ACCESSION.x" and "BIOSAMPLE_ACCESSION.y" parallel columns for same RESULT_VALUE_PREFERRED
drop = "BIOSAMPLE_ACCESSION.y"
res = results[,!(names(results) %in% drop)]

In [11]:
gene_mat_inner = wb_drop%>%inner_join(res, by=c("subject_accession"="SUBJECT_ACCESSION"))#<<== *KEEP THIS ONE*
#gene_mat = merge(wb,results, by=c("SUBJECT_ACCESSION"), all = TRUE)

In [12]:
head(gene_mat_inner)

,data_accession,subject_accession,age,gender,race,study_accession,EEF1A1,GAPDH,SLC35E2,DUSP22,⋯,RESULT_UNIT_PREFERRED,RESULT_UNIT_REPORTED,RESULT_VALUE_PREFERRED,RESULT_VALUE_REPORTED,WORKSPACE_ID,STUDY_ACCESSION,RESULT_ID,EXPSAMPLE_ACCESSION,REPOSITORY_ACCESSION,REPOSITORY_NAME
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<dbl>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>
1,Ji_Batch2_22.295185,SUB116449,69.4,Male,White,SDY112,11.07469,12.26052,6.594605,7.623319,⋯,,Titer,0.0,0,2883,SDY311,59,ES776616,GSM1422060,GEO
2,Ji_Batch2_90.295250,SUB116515,74.3,Female,White,SDY112,10.79645,11.71058,6.893926,7.698674,⋯,,Titer,-0.6,-0.6,2883,SDY315,509,ES781236,GSM1573176,GEO
3,Ji_Batch2_3.295166,SUB116516,85.7,Female,White,SDY112,11.21682,12.33463,6.735886,7.859069,⋯,,Titer,6.4,6.4,2883,SDY315,495,ES781254,GSM1573177,GEO
4,Ji_Batch2_52.295212,SUB116519,25.4,Male,White,SDY112,11.11929,11.79117,6.944692,7.462226,⋯,,Titer,-0.1,-0.1,2883,SDY315,510,ES781223,GSM1573178,GEO
5,Ji_Batch2_40.295201,SUB116520,79.1,Male,White,SDY112,11.05968,11.52055,6.958966,7.660048,⋯,,Titer,-0.1,-0.1,2883,SDY315,517,ES781274,GSM1573179,GEO
6,Ji_Batch2_48.295209,SUB116521,85.6,Female,White,SDY112,12.08360,12.06977,6.706719,7.670629,⋯,,Titer,5.6,5.6,2883,SDY315,506,ES781272,GSM1573180,GEO


In [13]:
gen_inner = gene_mat_inner[!duplicated(gene_mat_inner$subject_accession), ]

In [14]:
head(gen_inner)

,data_accession,subject_accession,age,gender,race,study_accession,EEF1A1,GAPDH,SLC35E2,DUSP22,⋯,RESULT_UNIT_PREFERRED,RESULT_UNIT_REPORTED,RESULT_VALUE_PREFERRED,RESULT_VALUE_REPORTED,WORKSPACE_ID,STUDY_ACCESSION,RESULT_ID,EXPSAMPLE_ACCESSION,REPOSITORY_ACCESSION,REPOSITORY_NAME
,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<chr>,<chr>,<dbl>,<chr>,<int>,<chr>,<int>,<chr>,<chr>,<chr>
1,Ji_Batch2_22.295185,SUB116449,69.4,Male,White,SDY112,11.07469,12.26052,6.594605,7.623319,⋯,,Titer,0.0,0,2883,SDY311,59,ES776616,GSM1422060,GEO
2,Ji_Batch2_90.295250,SUB116515,74.3,Female,White,SDY112,10.79645,11.71058,6.893926,7.698674,⋯,,Titer,-0.6,-0.6,2883,SDY315,509,ES781236,GSM1573176,GEO
3,Ji_Batch2_3.295166,SUB116516,85.7,Female,White,SDY112,11.21682,12.33463,6.735886,7.859069,⋯,,Titer,6.4,6.4,2883,SDY315,495,ES781254,GSM1573177,GEO
4,Ji_Batch2_52.295212,SUB116519,25.4,Male,White,SDY112,11.11929,11.79117,6.944692,7.462226,⋯,,Titer,-0.1,-0.1,2883,SDY315,510,ES781223,GSM1573178,GEO
5,Ji_Batch2_40.295201,SUB116520,79.1,Male,White,SDY112,11.05968,11.52055,6.958966,7.660048,⋯,,Titer,-0.1,-0.1,2883,SDY315,517,ES781274,GSM1573179,GEO
6,Ji_Batch2_48.295209,SUB116521,85.6,Female,White,SDY112,12.08360,12.06977,6.706719,7.670629,⋯,,Titer,5.6,5.6,2883,SDY315,506,ES781272,GSM1573180,GEO


In [15]:
cols = c('data_accession','subject_accession','age','gender','race','study_accession', 'LAB_TEST_ACCESSION','LAB_TEST_PANEL_ACCESSION','BIOSAMPLE_ACCESSION','NAME_PREFERRED','NAME_REPORTED','REFERENCE_RANGE_ACCESSION','RESULT_UNIT_PREFERRED','RESULT_UNIT_REPORTED','RESULT_VALUE_PREFERRED','RESULT_VALUE_REPORTED','WORKSPACE_ID','STUDY_ACCESSION','RESULT_ID','EXPSAMPLE_ACCESSION','REPOSITORY_ACCESSION','REPOSITORY_NAME')

In [40]:
class(summary(lm(gen_inner[, 'EEF1A1']~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION'])))#access via matrix notation

[1] "summary.lm"

In [57]:
summary(lm(gen_inner[, 'EEF1A1']~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION']))$coefficients[2,4]

[1] 0.5555223

In [61]:
summary(lm(gen_inner[, 'EEF1A1']~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION']))$coefficients['gen_inner[, "RESULT_VALUE_PREFERRED"]','Pr(>|t|)']

[1] 0.5555223

In [52]:
summary(lm(gen_inner[, 'EEF1A1']~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION']))$coefficients[17]

[1] 0.5555223

In [51]:
summary(lm(gen_inner[, 'EEF1A1']~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION']))$coefficients#can't access columns

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),11.44142832,0.37164213,30.7861444,1.488100e-48
"gen_inner[, ""RESULT_VALUE_PREFERRED""]",0.02288302,0.03866712,0.5917954,5.555223e-01
"gen_inner[, ""STUDY_ACCESSION""]SDY305",1.38054630,0.49483926,2.7898884,6.477752e-03
"gen_inner[, ""STUDY_ACCESSION""]SDY311",1.08700696,0.47383145,2.2940794,2.419464e-02
"gen_inner[, ""STUDY_ACCESSION""]SDY315",-0.34453133,0.38194263,-0.9020499,3.695209e-01


In [17]:
#gene_full$RESULT_VALUE_PREFERRED
gene_list = colnames(gen_inner)[!(colnames(gen_inner) == cols)]

In [18]:
posclass = vector()
negclass = vector()

In [19]:
# remember the environment issue

In [53]:
x = c('gene', 'class', 'cmvCoef', 'cmvSummPval','tMuPos', 'tMuNeg', 'tPval')
df = data.frame(matrix(ncol = 11, nrow = 0))
colnames(df) = x
# ****with or without intercept ??

In [54]:
# create function for add_row

In [55]:
for(gene in gene_list) {
    posclass = gen_inner[gene][gen_inner$RESULT_VALUE_PREFERRED > 2, ]
    negclass = gen_inner[gene][gen_inner$RESULT_VALUE_PREFERRED <= 2, ]
    
    stat = t.test(posclass, negclass)# default: unpaired, two sided
    diff = as.numeric(stat$estimate[1] - stat$estimate[2])# (posClassMu - negClassMu)

    # diff > 0; pos dominant
    if(diff > 0 & stat$p.value < 0.01) {
        add_row = data.frame(gene, 'c1', 
                             as.numeric(lm(gen_inner[, gene]~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION'])$coefficients[2]),
                             summary(lm(gen_inner[, gene]~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION']))$coefficients['gen_inner[, "RESULT_VALUE_PREFERRED"]','Pr(>|t|)'],
                             as.numeric(stat$estimate[1]), as.numeric(stat$estimate[2]), as.numeric(stat$p.value))
        colnames(add_row) = x
        df = rbind(df, add_row)
    }
    # diff < 0; neg dominant
    else if(diff  <= 0 & stat$p.value < 0.01) {
        add_row = data.frame(gene, 'c2', 
                             as.numeric(lm(gen_inner[, gene]~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION'])$coefficients[2]),
                             summary(lm(gen_inner[, gene]~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION']))$coefficients['gen_inner[, "RESULT_VALUE_PREFERRED"]','Pr(>|t|)'],
                             as.numeric(stat$estimate[1]), as.numeric(stat$estimate[2]), as.numeric(stat$p.value))
        colnames(add_row) = x
        df = rbind(df, add_row)
    }

    else if(stat$p.value > 0.01) {# non sig
        add_row = data.frame(gene, 'c3',
                             as.numeric(lm(gen_inner[, gene]~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION'])$coefficients[2]),
                             summary(lm(gen_inner[, gene]~gen_inner[,'RESULT_VALUE_PREFERRED']+gen_inner[,'STUDY_ACCESSION']))$coefficients['gen_inner[, "RESULT_VALUE_PREFERRED"]','Pr(>|t|)'],
                             as.numeric(stat$estimate[1]), as.numeric(stat$estimate[2]), as.numeric(stat$p.value))
        colnames(add_row) = x
        df = rbind(df, add_row)
    }
}

In [56]:
head(df)

,gene,class,cmvCoef,cmvSummPval,tMuPos,tMuNeg,tPval
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,EEF1A1,c3,0.022883024,0.5555223,11.636491,11.416069,0.3789520
2,GAPDH,c3,0.001589537,0.9089341,11.996509,11.957632,0.6699054
3,SLC35E2,c3,0.010216179,0.3351051,7.021055,6.960301,0.3880087
4,DUSP22,c3,0.011816785,0.3987745,7.770124,7.686246,0.3395835
5,RPS28,c3,0.030781268,0.1454415,12.580419,12.417138,0.2040993
6,IPO13,c3,0.012574863,0.4829771,5.428254,5.351064,0.5244349


In [ ]:
#fwrite(df, paste0("gene_statistics" ,".csv"))